In [68]:
import numpy as np
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
from IPython.display import HTML
from sklearn.preprocessing import StandardScaler
np.random.seed(0)

# Parameters
L = 20  # Length of domain
N = 256  # Number of grid points
dx = L / N  # Grid spacing
x = np.linspace(0, L, N, endpoint=False)  # Spatial grid

# Function to create sinusoidal initial condition
def create_initial_condition():
    k1, k2 = 2, 1  # Wavenumbers
    phase1, phase2 = np.random.rand() * 2 * np.pi, np.random.rand() * 2 * np.pi
    amplitude1, amplitude2 = np.random.rand(), np.random.rand() * 0.5
    return amplitude1 * np.sin(2 * np.pi * k1 * x / L + phase1) + amplitude2 * np.sin(2 * np.pi * k2 * x / L + phase2)

# Define the Kuramoto-Sivashinsky equation
def kuramoto_sivashinsky(t, u):
    uxx = np.roll(u, 1) - 2 * u + np.roll(u, -1)
    uxxxx = np.roll(u, 2) - 4 * np.roll(u, 1) + 6 * u - 4 * np.roll(u, -1) + np.roll(u, -2)
    dudt = - uxx - uxxxx - 0.5 * (np.roll(u, -1) * np.roll(u, 1) - np.roll(u, -1) * np.roll(u, -2))
    return dudt

# Time grid
t_span = np.linspace(0, 30, 20)

# Solve the ODE system for multiple initial conditions
all_solutions = []
num_initial_conditions = 500

for _ in range(num_initial_conditions):
    u0 = create_initial_condition()
    sol = solve_ivp(kuramoto_sivashinsky, [t_span[0], t_span[-1]], u0, t_eval=t_span, method='RK45')
    all_solutions.append(sol.y.T)

all_solutions = np.array(all_solutions)
print(all_solutions.shape)

scaler = StandardScaler()
all_solutions = scaler.fit_transform(all_solutions.reshape(-1, N)).reshape(num_initial_conditions, len(t_span), N)

(500, 20, 256)


In [69]:
from sklearn.model_selection import train_test_split

nn_input = all_solutions[:, 0]
nn_output = all_solutions[:, 1:]
X_train, X_test, Y_train, Y_test = train_test_split(nn_input, nn_output, test_size=0.3, shuffle=False)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(350, 256) (350, 19, 256) (150, 256) (150, 19, 256)


In [70]:
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LSTM, RepeatVector, Flatten, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

# TODO: Add attention to this model
def create_model(learning_rate, lstm_units, dropout_rate, N, T_out):
    model = Sequential()
    model.add(Input(shape=(N,)))  # Single input vector of size N
    model.add(RepeatVector(T_out))  # Repeat the input vector T_out times to create a sequence
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))  # Use Bidirectional LSTM
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))  # Another Bidirectional LSTM layer
    model.add(Dropout(dropout_rate))  # Another dropout layer
    model.add(Dense(N))  # Output sequence with each time step having N features
    model.add(Flatten())
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [71]:
model = KerasRegressor(model=create_model, verbose=1, N=N, T_out=len(t_span) - 1, batch_size=38, epochs=80, learning_rate=1e-3, lstm_units=256, dropout_rate=0.2)

# Fit the BayesSearchCV
best_model = model.fit(X_train, Y_train.reshape(Y_train.shape[0], -1))

Epoch 1/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - loss: 0.7979
Epoch 2/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.4132
Epoch 3/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.3147
Epoch 4/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.2810
Epoch 5/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.2453
Epoch 6/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 0.2302
Epoch 7/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.2170
Epoch 8/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.2141
Epoch 9/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.1920
Epoch 10/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - loss: 0.1791
Epoch 11/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.1940
Epoch 12/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.1689
Epoch 13/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - loss: 0.1675
Epoch 14/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - loss: 0.1680
Epoch 15/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 0.160

In [76]:
score = best_model.score(X_test, Y_test.reshape(Y_test.shape[0], -1))
print(f'Score: {score}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Score: 0.9011465167511882


In [83]:
np.random.seed(10)
u0 = create_initial_condition()
print(u0.shape)
sol = best_model.predict(u0.reshape(1, -1)).reshape(-1, N)

(256,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [84]:
act_sol = solve_ivp(kuramoto_sivashinsky, [t_span[0], t_span[-1]], u0.flatten(), t_eval=t_span, method='RK45').y.T[:-1]
print("Network output", sol.shape)
print("Actual solution", act_sol.shape)

Network output (19, 256)
Actual solution (19, 256)


In [85]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
line, = ax.plot(x, act_sol[0], label='Actual')
line2, = ax.plot(x, sol[0], label='Predicted')
plt.close()

def update(frame):
    line.set_ydata(act_sol[frame])
    line2.set_ydata(sol[frame])
    return line, line2,

ani = FuncAnimation(fig, update, frames=len(t_span) - 1, blit=True, interval=100)
HTML(ani.to_jshtml())